In [1]:
# Setting up the environment.
import numpy as np
import pandas as pd
from scipy import stats

In [2]:
# Load the data from JHU
df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
df_d = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')

In [3]:
df.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20
0,16.0,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316.0,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,141,141,141,141,145,145,145,145,145,145
2,580.0,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,14,14,14,14,14,14,14,14,14,14


In [4]:
US = df[(df["Country_Region"] == "US")]
US_d = df_d[(df_d["Country_Region"] == "US")]

In [5]:
US.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20
0,16.0,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316.0,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,141,141,141,141,145,145,145,145,145,145
2,580.0,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,14,14,14,14,14,14,14,14,14,14


In [6]:
dateColumns = US.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2' ,'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key'], axis=1)
dates = dateColumns.columns
dates

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '4/26/20', '4/27/20', '4/28/20', '4/29/20', '4/30/20', '5/1/20',
       '5/2/20', '5/3/20', '5/4/20', '5/5/20'],
      dtype='object', length=105)

In [7]:
US_states = US.groupby(['Province_State']).sum()
US_states = US_states.loc[:, '1/22/20':]
US_states = US_states.sort_values(by='Province_State', ascending=False)
US_states.head(5)

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20
Province_State,,,,,,,,,,,,,,,,,,,,,
Wyoming,0,0,0,0,0,0,0,0,0,0,...,502,520,536,545,559,566,579,586,596,604
Wisconsin,0,0,0,0,0,0,0,0,0,0,...,5912,6081,6289,6520,6854,7314,7660,7964,8236,8566
West Virginia,0,0,0,0,0,0,0,0,0,0,...,1055,1063,1095,1110,1125,1152,1185,1195,1224,1246
Washington,1,1,1,1,1,1,1,1,1,1,...,13521,13686,13842,14070,14327,14637,15003,15185,15462,15594
Virginia,0,0,0,0,0,0,0,0,0,0,...,12970,13538,14339,14962,15847,16902,17738,18672,19493,20257


In [8]:
US_states_d = US_d.groupby(['Province_State']).sum()
US_states_d = US_states_d.loc[:, '1/22/20':]
US_states_d = US_states_d.sort_values(by='Province_State', ascending=False)

In [9]:
US_states_d.head(3)

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20
Province_State,,,,,,,,,,,,,,,,,,,,,
Wyoming,0,0,0,0,0,0,0,0,0,0,...,7,7,7,7,7,7,7,7,7,7
Wisconsin,0,0,0,0,0,0,0,0,0,0,...,272,281,300,308,316,327,334,339,340,353
West Virginia,0,0,0,0,0,0,0,0,0,0,...,34,34,38,38,41,46,48,48,48,48


In [10]:
def daily_DF(df):
    # this cell by Leaf Petersen
    # Copy the data frame
    dfdaily=df.copy()

    # We are on the first iteration
    first=True
    # For every column
    for column in df:
       # If this is the first iteration
       if first:
         # Save the first column
         prev = column
         # Copy over the first column unchanged
         dfdaily[column]=df[column]
         # All subsequent iterations are not the first
         first=False
         # Skip the rest of the loop
         continue
       # We are not the first iteration, so keep going
       # Set the new column contents to the difference of the
       # previous column and the current column
       dfdaily[column] = df[column]-df[prev]
       # The current column is the previous column for the next iteration
       prev = column
        
#     # We only want the totals
#     dfdaily = dfdaily.sum().to_frame()
#     dfdaily = dfdaily.reset_index()
#     # Negatives into positives
#     dfdaily[0] = dfdaily[0].abs()
    
    return(dfdaily)

In [11]:
US_States_daily = daily_DF(US_states)
US_States_daily_d = daily_DF(US_states_d)

In [12]:
US_States_daily_d.head(10)

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20
Province_State,,,,,,,,,,,,,,,,,,,,,
Wyoming,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Wisconsin,0,0,0,0,0,0,0,0,0,0,...,6,9,19,8,8,11,7,5,1,13
West Virginia,0,0,0,0,0,0,0,0,0,0,...,2,0,4,0,3,5,2,0,0,0
Washington,0,0,0,0,0,0,0,0,0,0,...,12,15,22,15,13,10,6,4,7,21
Virginia,0,0,0,0,0,0,0,0,0,0,...,12,11,32,30,30,29,36,43,24,29
Virgin Islands,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Vermont,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,2,1,1,2,-1,0
Utah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,4,1,0,3,1,0,6
Texas,0,0,0,0,0,0,0,0,0,0,...,13,15,53,35,58,28,23,15,23,24


In [13]:
values = US_States_daily[:].replace(0, np.nan)
values.tail(25)

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20
Province_State,,,,,,,,,,,,,,,,,,,,,
Maryland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,815.0,906.0,626.0,736.0,893.0,1730.0,1001.0,989.0,946.0,709.0
Maine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,50.0,8.0,17.0,16.0,39.0,28.0,30.0,32.0,20.0,21.0
Louisiana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,261.0,295.0,218.0,374.0,341.0,710.0,429.0,200.0,333.0,323.0
Kentucky,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,170.0,71.0,219.0,162.0,171.0,174.0,-3.0,251.0,115.0,577.0
Kansas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,142.0,196.0,179.0,187.0,574.0,221.0,251.0,271.0,227.0,265.0
Iowa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,384.0,392.0,508.0,467.0,302.0,739.0,759.0,532.0,528.0,408.0
Indiana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,613.0,949.0,627.0,594.0,653.0,795.0,665.0,638.0,574.0,526.0
Illinois,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,2126.0,1980.0,2219.0,2256.0,2560.0,3137.0,2450.0,2994.0,2278.0,2112.0
Idaho,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,NaN,55.0,NaN,32.0,31.0,46.0,NaN,45.0,21.0


In [14]:
import plotly.graph_objects as go
import datetime
import numpy as np

z=values
x=dates
y=values.index #the sates

d=US_States_daily_d #deaths for the tooltip:.3f
f=US_States_daily_d
customdata = np.dstack((d, f))

fig = go.Figure(data=go.Heatmap(
        z=z,
        x=x,
        y=y,
        customdata=np.dstack((d, f)),
        hovertemplate='State: %{y}<br>Confirmed: <b> %{z:,.0f} </b> <br>Deaths: <b> %{customdata[0]:,.0f} </b> <extra></extra>',
        colorscale='matter',
        xgap=1,
        ygap=12,
        zmin=0,
        zmid=2500,
        zmax=5000,
        colorbar=dict(thickness=10, title='Confirmed <br>_', separatethousands=True),
        connectgaps=False
               )
        )

fig.update_layout(
        title='Daily cases per state',
        title_x=0.5,
        xaxis_nticks=18,
        yaxis_nticks=64,
        height=1800,
        paper_bgcolor='white', 
        plot_bgcolor='#ffffe6',
        hovermode="x unified",
        hoverlabel=dict(
                bgcolor="white", 
                font_size=16, 
                font_family="Arial",
                bordercolor='#b3b3b3'),
        dragmode=False) #disable panning so mobile apps can receive pinch to zoom events
    
fig.update_xaxes(side='top', fixedrange=True, 
                 showgrid=False,
                 spikethickness=0.4,
                 spikedash='solid',)

fig.update_yaxes(gridcolor='#ffe6cc')

fig.show(displayModeBar=False)
    
import plotly.io as pio
pio.write_html(fig, file='Index.html', auto_open=True)